In [16]:
import os
import pandas as pd
import numpy as np
import json

In [4]:
read_dir='E:/文档/数据集/Data Sets used in Computing Education/ASSISTment Data Sets/ASSISTment Data/2009-2010 ASSISTment Data/Skill-builder data 2009-2010/'
save_dir='../data/a0910/'

prob_count_limit=15 #每个学生做过的习题下限
data_set=pd.read_csv(read_dir+'skill_builder_data_corrected.csv',encoding='ISO-8859-1')

In [11]:
data=data_set.loc[:,['user_id','problem_id','correct','skill_name']]\
    .drop_duplicates(subset=['user_id','problem_id','skill_name'],keep='first').dropna(axis=0,how='any')
data.columns=['user_id','item_id','score','knowledge_code']

In [12]:
record=data.loc[:,['user_id','item_id','score']].drop_duplicates(subset=['user_id','item_id'],keep='first').dropna(axis=0,how='any')
item_conc_data=data.loc[:,['item_id','knowledge_code']].drop_duplicates(keep='first').dropna(axis=0,how='any')

In [13]:
# 统计每个学生做了多少道题
problem_counter=record.groupby(by='user_id').count()

filtered_stu_boundary=prob_count_limit #学生做的题超过15道才算数

filtered_stu_id=problem_counter[problem_counter['item_id']>filtered_stu_boundary].index.to_numpy()

print('有效的学生数：',len(filtered_stu_id))

record=record.set_index('user_id').loc[filtered_stu_id,:].reset_index()

有效的学生数： 2380


In [14]:
item_unique=np.unique(record['item_id'])
know_unique=np.unique(item_conc_data['knowledge_code'])
stu_unique=np.unique(record['user_id'])

In [15]:
stu_old_new=dict(zip(stu_unique,range(1,len(stu_unique)+1)))
item_old_new=dict(zip(item_unique,range(1,len(item_unique)+1)))
know_old_new=dict(zip(know_unique,range(1,len(know_unique)+1)))

In [17]:
with open(save_dir+'dict_knowledge_code.json','w') as f:
    json.dump(know_old_new,f)

In [18]:
record['user_id']=record['user_id'].map(stu_old_new)
record['item_id']=record['item_id'].map(item_old_new)

item_conc_data['item_id']=item_conc_data['item_id'].map(item_old_new)
item_conc_data['knowledge_code']=item_conc_data['knowledge_code'].map(know_old_new)

print('Done')

Done


In [19]:
item_open_df=item_conc_data.set_index('item_id')

item_df=pd.DataFrame(columns=['item_id','knowledge_code'],index=range(1,len(item_unique)+1))
for item in range(1,len(item_unique)+1):
    item_df.loc[item,'item_id']=item
    item_df.loc[item,'knowledge_code']=np.array(item_open_df.loc[item,['knowledge_code']]).reshape(-1).tolist()

In [20]:
item_df.to_csv(save_dir+'item.csv',index=False)
record.to_csv(save_dir+'record.csv',index=False)

In [21]:
print('学习者数：',len(stu_unique))
print('习题数：',len(item_unique))
print('知识点数：',len(know_unique))
print('记录数：',len(record))

学习者数： 2380
习题数： 16804
知识点数： 110
记录数： 257585
